In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Paths Input Here
analysis = "memory_network_masked_to_minima"
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    unmasked_networks = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/misc_analyses/overlap_of_f_stat_with_memory_network/stn_dbs_f_stat_95th_percentile.nii'    
    mask_nifti = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/rois/memory_roi.nii'
    out_dir = rf'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/misc_analyses/overlap_of_f_stat_with_memory_network/{analysis}'
    #out_dir = r'path to out dir here'
    print('I have set pathnames in the Mac style')
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    conn_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_AvgR.csv'
    clin_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\patient_data\AD_Clinical_Data_CDR_ADAS_COG_13.xlsx'
    # clin_path = 'path to clinical values'
    out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\stats'
    #out_dir = r'path to out dir here'
    x_roi_names = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\matrix_corrMx_names.csv'
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Windows style')

In [ ]:
from natsort import natsorted

file_pattern = ''
glob_path  = unmasked_networks + file_pattern
print('I will search: ', glob_path)

globbed = glob.glob(glob_path)
#Identify files of interest
matrix_df = pd.DataFrame({})
names = []
for file in globbed:
    print('I found : ', file)
    img = image.load_img(file)
    #Organize files into manipulatable dataframes
    data = img.get_fdata(); data = np.nan_to_num(data, nan=0, posinf=10, neginf=-10)

    #Organize the names of the files into something understandable
    name = os.path.basename(file).split('_tome')[0]
    try:
        name = name.split('sub-')[1]
        name = name.split('uvat')[0]
    except:
        print('cannot further split name')
    matrix_df[name] = data.flatten()
    names.append(name)
matrix_df = matrix_df.reindex(columns=natsorted((list(matrix_df))))

In [ ]:
matrix_df.tail(5)

In [ ]:
#Perform Masking to the Mask Nifti
threshold_df = matrix_df.copy()
print('Number prior voxels: ', np.count_nonzero(threshold_df))
mask_img = image.load_img(mask_nifti); mask_data = mask_img.get_fdata(); mask_data = np.nan_to_num(mask_data, nan=0, posinf=0, neginf=0)
flat_mask = mask_data.flatten()
threshold_df[flat_mask == 0] = 0

print('Number surviving voxels: ', np.count_nonzero(threshold_df))
threshold_df.tail(5)

In [ ]:
#Ensure Success
mask = nimds.get_img("mni_icbm152")
print(mask_data.shape)
img_mx = np.reshape(np.array(threshold_df.iloc[:,0]), mask_data.shape)
ovr_img1 = image.new_img_like(mask, img_mx)
ovr_html1 = plotting.view_img(ovr_img1, cut_coords=(0,0,0), title=('Test Image'), black_bg=False, opacity=.75, cmap='ocean_hot')
ovr_html1

In [ ]:
#Save the Masked Files
if os.path.isdir(out_dir)==False:
    os.mkdir(out_dir)
    
for i in range(0, threshold_df.shape[1]):
    matrix = np.reshape(np.array(threshold_df.iloc[:,i]), mask_img.shape)
    img = image.new_img_like(mask, matrix)
    img.to_filename(os.path.join(out_dir, threshold_df.columns[i]))
print('saved to: ', out_dir)